# Small World Graphs

Code examples from [Think Complexity, 2nd edition](https://thinkcomplex.com).

Copyright 2016 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns
from collections import deque

from utils import decorate, savefig

# I set the random seed so the notebook 
# produces the same results every time.
np.random.seed(17)

In [ ]:
# node colors for drawing networks
colors = sns.color_palette('pastel', 5)
#sns.palplot(colors)
sns.set_palette(colors)

**Exercise 3.4:** In the book, I claimed that Dijkstra's algorithm does not work unless it uses BFS.  Write a version of `shortest_path_dijkstra` that uses DFS and test it on a few examples to see what goes wrong.

In [ ]:
def shortest_path_dfs(G, source):
    #Modified shortest_path_dijkstra that uses a dfs instead of a bfs
    #In the book, performing a BFS uses the first element (pop(0) or popleft()) is used.
    #We replace that line to turn the BFS implementation into a DFS.
    dist = {source: 0}
    queue = deque([source])
    while queue:
        node = queue.pop()
        new_dist = dist[node] + 1

        neighbors = set(G[node]).difference(dist)
        for n in neighbors:
            dist[n] = new_dist
        
        queue.extend(neighbors)
    return dist

Let's test this implementation to measure the path lengths in a regular graph with 10 nodes each with 6 nearest neighors

In [ ]:
#From exercise 3.1, we use this to generate graphs for testing

#Function from the notebook
def adjacent_edges(nodes, halfk):
    """Yields edges between each node and `halfk` neighbors.
    
    halfk: number of edges from each node
    """
    n = len(nodes)
    for i, u in enumerate(nodes):
        for j in range(i+1, i+halfk+1):
            v = nodes[j % n]
            yield u, v
            
            
def opposite_edge(nodes):
    n = len(nodes)
    halfn = n//2
    
    #We take the opposite node (for even number of nodes) by taking half the length of the list
    #and using that to index the nodes. Taking the modulon guarantees that you are periodically 
    #indexing within the length the list
    
    for i, u in enumerate(nodes):
        index = (i + halfn)%n
        v = nodes[index]
        yield u,v
        


def make_regular_graph(n, k):
    #This function uses adjacent_edges and opposite_edge (when k is odd)
    #Getting the remainder and the quotient is used for checking if
    #k is odd or not.
    
    #A regular graph is a graph that has nodes that have the same number of neighbors
    #In this case, n is the number of nodes and k is the number of neighbors.
    
    quo_k = k//2
    mod_k = k%2
    
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(adjacent_edges(nodes, quo_k))
    
    if mod_k == 1: #when k is odd
        if n%2 == 0: #when nodes are even
            G.add_edges_from(opposite_edge(nodes))
        else:
            raise ValueError("Regular graph cannot be generated if both n and k are both odd.")
            
    return G

#From exercise 3.2, we use this to compare results with the 'faulty' shortest path function
def shortest_path_length_mod(G, start):
    #Modified plain_bfs to get path lengths instead
    #G is the graph and start is the starting node.
    
    dist_counter = 0 #Starting distance from start
    dist = {}
    nextlevel = {start} #Initialize while loop with the starting node
    
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        for v in thislevel:
            if v not in dist: #only execute if v is not yet in the list of distances
                dist[v] = dist_counter #if the v is not in the dictionary
                nextlevel.update(G[v]) #Update set with neighbors of v
        dist_counter += 1 #add 1 to distance
    
    return dist

In [ ]:
graph = make_regular_graph(10, 6)

In [ ]:
nx.draw_circular(graph, 
                 node_color='C1', 
                 node_size=1000, 
                 with_labels=True)

In [ ]:
dist_dfs = shortest_path_dfs(graph, 0)

In [ ]:
dist_bfs = shortest_path_length_mod(graph, 0)

In [ ]:
dist_nx = nx.shortest_path_length(graph, 0)

In [ ]:
dist_dfs == dist_bfs

In [ ]:
dist_dfs == dist_nx

In [ ]:
dist_bfs == dist_nx

We see that only the distance with the dfs implementation of shortest path gets the wrong answer.